# Search for long range edges 

In this notebook, I will implement code to search for a long range edge(s) from a default `feems` by seeing if the likelihood increases with the addition of a certain edge. First, the search for the *best* edge will be done through a full grid search, wherein I iteratively add one long range edge after another over all pairs of nodes and check if this extra edge decreases the negative log likelihood. Second, I will implement a heuristic search by using a greedy approach to fit an edge between a pair of nodes that show maximum residuals with the default fit. 

To do this, I need to simulate a large empirical test case with many nodes. I will use the same corridor-barrier-corridor approach from previous simulations but with a much larger grid (40x80 is too large, for instance, 10x50 took 2 days to only get through ~1% of all edges). Keep total number of sampled nodes to about 40 maybe? 

## Changes to original code base

1. added code to calculate the negative log likelihood value for fit in `spatial_graph.py` (basically, adding `Objective` functions)

## Imports 

In [185]:
%load_ext autoreload
%autoreload 2

# base
import numpy as np
import networkx as nx
from sklearn.impute import SimpleImputer
import pkg_resources
import itertools as it
import math
from scipy.spatial.distance import pdist, squareform
import statsmodels.api as sm
from copy import deepcopy
import pandas as pd
import multiprocessing as mp

# viz
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cartopy.crs as ccrs

# feems
from feems.utils import prepare_graph_inputs
from feems import SpatialGraph, Viz, Objective
from feems.sim import setup_graph, setup_graph_long_range, simulate_genotypes
from feems.spatial_graph import query_node_attributes
from feems.objective import comp_mats
from feems.cross_validation import run_cv
from feems.helper_funcs import plot_default_vs_long_range, comp_genetic_vs_fitted_distance, plot_estimated_vs_simulated_edges

# change matplotlib fonts
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.sans-serif"] = "Arial"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Simulation test case

### Exhaustive search 

This requires a search over a little less than $\sim \frac{d^2}{2}$ edges (for 128 nodes, it is about 7,800 edges). Using some kind of multithreading since this problem is embarassingly parellel. 


In [159]:
n_rows, n_columns = 8, 16
graph_def, _, _, edge_def = setup_graph(n_rows=n_rows, n_columns=n_columns, barrier_startpt=2.5, barrier_endpt=6.5, corridor_w=0.5, barrier_w=0.1, barrier_prob=1.0)

lrn = [(0,38)]

## using 1.0 to ensure all nodes are sampled equally well (default params otherwise: 4x8 grid)
graph, coord, grid, edge = setup_graph_long_range(n_rows=n_rows, n_columns=n_columns, corridor_w=1.0, barrier_w=0.5, barrier_prob=1.0, long_range_nodes=lrn, long_range_edges=[1.5])

gen_test = simulate_genotypes(graph)

Simulating ~SNP 0
Simulating ~SNP 50
Simulating ~SNP 100
Simulating ~SNP 150
Simulating ~SNP 200
Simulating ~SNP 250
Simulating ~SNP 300
Simulating ~SNP 350
Simulating ~SNP 400
Simulating ~SNP 450
Simulating ~SNP 500
Simulating ~SNP 550
Simulating ~SNP 600
Simulating ~SNP 650
Simulating ~SNP 700
Simulating ~SNP 750
Simulating ~SNP 800
Simulating ~SNP 850
Simulating ~SNP 900
Simulating ~SNP 950


In [160]:
sp_Graph_def = SpatialGraph(gen_test, coord, grid, edge_def)

In [161]:
# create a list of all edges to add (since it is symmetric, we should expect d(d-1)/2 but some will be adjacent nodes so fewer than that)
lr = (tuple(i) for i in it.product(tuple(range(sp_Graph_def.n_observed_nodes)), repeat=2) if tuple(reversed(i)) > tuple(i))
final_lr = [x for x in list(lr) if x not in list(sp_Graph_def.edges)]

In [162]:
df = pd.DataFrame(index = np.arange(len(final_lr)), columns = ['nodes', 'nll'])

In [176]:
# function that takes in an edge, adds to default graph and returns negative log-lik
def add_edge(val):
    edges_lr = deepcopy(edge_def)
    edges_lr = edges_lr.tolist()
    edges_lr.append(list(x+1 for x in val))
    sp_Graph = SpatialGraph(gen_test, coord, grid, np.array(edges_lr))
    sp_Graph.fit(lamb = 10.0, verbose=False)
    return sp_Graph.nll

In [163]:
%%time
# for idx, val in enumerate(final_lr):
#     # creating a new edge vector (all default edges + 1 long range)
#     edges_lr = deepcopy(edge_def)
#     edges_lr = edges_lr.tolist()
#     edges_lr.append(list(x+1 for x in val))
#     sp_Graph = SpatialGraph(gen_test, coord, grid, np.array(edges_lr))
#     sp_Graph.fit(lamb = 10.0, verbose=False)

#     df.iloc[idx, 0] = val
#     df.iloc[idx, 1] = sp_Graph.nll

# tbh no difference in timings between above for loop and one-liner map
df['nodes'] = final_lr
df['nll'] = list(map(add_edge, df.iloc[np.arange(len(final_lr)),0]))

# print nodes connected by THE edge to give lowest negative log likelihood
df.loc[df['nll'].astype(float).idxmin(),'nodes']

CPU times: user 7h 25min 18s, sys: 4min 15s, total: 7h 29min 34s
Wall time: 3h 39min 5s


(0, 38)

In [177]:
# %%time
# # sequential run for timing purposes
# df_test = pd.DataFrame(index = np.arange(100), columns = ['nodes', 'nll'])
# for idx, val in enumerate(final_lr[0:100]):
#     df_test.iloc[idx, 0] = val
#     df_test.iloc[idx, 1] = add_edge(val)

CPU times: user 5min 33s, sys: 4.04 s, total: 5min 37s
Wall time: 2min 48s


In [183]:
df_test

,nodes,nll
0,"(0, 2)",13988.335454
1,"(0, 3)",13951.909591
2,"(0, 4)",13895.398119
3,"(0, 5)",13854.259583
4,"(0, 6)",13861.783873
...,...,...
95,"(0, 98)",13950.784067
96,"(0, 99)",13976.136087
97,"(0, 100)",13929.84991
98,"(0, 101)",13870.904828


In [214]:
%%time
df_test['nodes'] = df.iloc[0:100,0]
df_test['nll'] = list(map(add_edge, df_test.iloc[0:100,0]))

CPU times: user 5min 2s, sys: 30.9 s, total: 5min 33s
Wall time: 2min 49s


In [213]:
# %%time
# # this section of code takes forever, some overhead in communication potentially
# pool = Pool(processes=mp.cpu_count())

# nll = pool.map(add_edge, (val for val in df_test['nodes']))

# pool.close()
# pool.join()

# df_test['nll'] = nll

KeyboardInterrupt: 

Based on initial testing on the simulations, I find that an exhaustive search works well in recovering the long range edge based on maximum decrease in negative log-likelihood. 

### Implementing a heuristic seach

We can use a data-driven apporach as before to find the pairs of nodes with highest residuals, and then search in a set region around the node. For instance, let's say, we get back (1,40) as the pair of nodes. We then implement a search in which we add edges between (0,40), (1,40), (2,40), ..., (2,41), etc. In this simple scheme, I will choose the six closest neighbors (forming a hexagon, if an interior node) to the selected nodes in the pair. 

In [218]:
# obtaining pairs of nodes with maximum residuals (TOP 1)
max_res_nodes = comp_genetic_vs_fitted_distance(sp_Graph_def, n_lre=1, lamb=10.0, plotFig=False)

constant-w/variance fit, converged in 123 iterations, train_loss=13990.5887471
lambda=10.0000000, alpha=0.6633522, converged in 14 iterations, train_loss=12458.5572596


ValueError: not enough values to unpack (expected 2, got 1)

In [269]:
# extracting closest neighbors to each node with maximum residuals
n1 = list(sp_Graph_def[max_res_nodes[0][0]])
n2 = list(sp_Graph_def[max_res_nodes[0][1]])

n1.append(max_res_nodes[0][0])
n2.append(max_res_nodes[0][1])

In [283]:
lr_heur = (tuple(i) for i in it.product(n1, n2))
# removing nodes that are already connected in the default graph 
# TODO: introduce a measure here to remove nodes that are close by on the graph
final_lr_heur = [x for x in list(lr_heur) if x not in list(sp_Graph_def.edges)]

In [310]:
%%time
df_heur = pd.DataFrame(index = np.arange(len(final_lr_heur)), columns = ['nodes', 'nll'])

df_heur['nodes'] = final_lr_heur
df_heur['nll'] = list(map(add_edge, df_heur.iloc[np.arange(len(final_lr_heur)),0]))

# print nodes connected by THE edge to give lowest negative log likelihood
df_heur.loc[df_heur['nll'].astype(float).idxmin(),'nodes']

CPU times: user 1min 2s, sys: 6.79 s, total: 1min 9s
Wall time: 35.5 s


(0, 38)

### Code to create a convex hull around nodes

Below I will write some code to create a search region around the selected nodes in which I will incorporate nodes to use in the search. Important use cases to keep in mind:  
1. ensure that the two convex hulls are non-overlapping (i.e., at least 2 neighbors apart)  
2. set a manual threshold if the closest sampled node is too far  

#### Simulating a use-case different from the previous 